In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [5]:
bets = pd.read_csv('In_Season/Data/todays_bets.csv', index_col = 0)
# bets.drop([6], inplace = True)
# bets.reset_index(inplace = True, drop = True)
# bets.to_csv('In_Season/Data/todays_bets.csv')

In [17]:
results = pd.read_csv('In_Season/Data/results_tracker.csv', index_col = 0)
# results

In [6]:
# Map for 538 scraping
fivethirtyeight_team_map =  {
    'Pistons' : 'Detroit Pistons',
    'Thunder' : 'Oklahoma City Thunder',
    'Kings' : 'Sacramento Kings',
    'Trail Blazers' : 'Portland Trailblazers',
    'Spurs' : 'San Antonio Spurs',
    'Raptors' : 'Toronto Raptors',
    'Rockets' : 'Houston Rockets',
    'Magic' : 'Orlando Magic',
    'Warriors' : 'Golden State Warriors',
    'Celtics' : 'Boston Celtics',
    'Cavaliers' : 'Cleveland Cavaliers',
    'Mavericks' : 'Dallas Mavericks',
    'Hornets' : 'Charlotte Hornets',
    'Pacers' : 'Indiana Pacers',
    'Grizzlies' : 'Memphis Grizzlies',
    'Wizards' : 'Washington Wizards',
    'Knicks' : 'New York Knicks',
    'Nets' : 'Brooklyn Nets',
    'Bucks' : 'Milwaukee Bucks',
    'Bulls' : 'Chicago Bulls',
    'Pelicans' : 'New Orleans Pelicans',
    'Jazz' : 'Utah Jazz',
    'Nuggets' : 'Denver Nuggets',
    'Clippers' : 'Los Angeles Clippers',
    'Lakers' : 'Los Angeles Lakers',
    '76ers' : 'Philadelphia 76ers',
    'Hawks' : 'Atlanta Hawks',
    'Heat' : 'Miami Heat',
    'Timberwolves' : 'Minnesota Timberwolves',
    'Suns' : 'Phoenix Suns'
}

In [15]:
# Scraping data from 538
tables = pd.read_html(f'https://projects.fivethirtyeight.com/{current_year}-nba-predictions/games/')
fivethirtyeight_data = pd.DataFrame(columns = ['Date', 'Away_Team', 'Away_Prob', 'Home_Team', 'Home_Prob'])
for i, table in enumerate(tables):
    if i == 0:
        continue
    if i > 30:
        break
    if i % 2 == 0:
        matchup_data = table.iloc[:, [2,4]]
        matchup_data.drop(2, axis = 0, inplace = True)
        matchup_data.columns = ['Teams', 'Probability']
        away_team = matchup_data.loc[0, 'Teams']
        away_prob = matchup_data.loc[0, 'Probability']
        home_team = matchup_data.loc[1, 'Teams']
        home_prob = matchup_data.loc[1, 'Probability']
        date = dt.date.today()
        data_series = pd.Series([date, away_team, away_prob, home_team, home_prob], index = fivethirtyeight_data.columns)
        fivethirtyeight_data = fivethirtyeight_data.append(data_series, ignore_index = True)

# Changing team names using team map
fivethirtyeight_data['Home_Team'] = fivethirtyeight_data.Home_Team.apply(lambda x: team_map[x])
fivethirtyeight_data['Away_Team'] = fivethirtyeight_data.Away_Team.apply(lambda x: team_map[x])

/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
# Retreiving odds


# Creating function to convert odds to probability
def calculate_odds(odds):
    if odds<0:
        return (abs(odds)/(abs(odds)+100))*100
    if odds>0:
        return (100/(odds+100))*100

# Creating team map to align w/ final output
team_map = {
    'Wizards' : 'Washington Wizards',
    'Celtics' : 'Boston Celtics',
    'Pelicans' : 'New Orleans Pelicans',
    'Knicks' : 'New York Knicks',
    'Pistons' : 'Detroit Pistons',
    'Magic' : 'Orlando Magic',
    '76ers' : 'Philadelphia 76ers',
    'Hawks' : 'Atlanta Hawks',
    'Pacers' : 'Indiana Pacers',
    'Raptors' : 'Toronto Raptors',
    'Grizzlies' : 'Memphis Grizzlies',
    'Heat' : 'Miami Heat',
    'Bulls' : 'Chicago Bulls',
    'Jazz' : 'Utah Jazz',
    'Bucks' : 'Milwaukee Bucks',
    'Spurs' : 'San Antonio Spurs',
    'Warriors' : 'Golden State Warriors',
    'Thunder' : 'Oklahoma City Thunder',
    'Timberwolves' : 'Minnesota Timberwolves',
    'Nuggets' : 'Denver Nuggets',
    'Suns' : 'Phoenix Suns',
    'Cavaliers' : 'Cleveland Cavaliers',
    'Mavericks' : 'Dallas Mavericks',
    'Kings' : 'Sacramento Kings',
    'Hornets' : 'Charlotte Hornets',
    'Trail Blazers' : 'Portland Trailblazers',
    'Nets' : 'Brooklyn Nets',
    'Lakers' : 'Los Angeles Lakers',
    'Clippers' : 'Los Angeles Clippers',
    'Rockets' : 'Houston Rockets'
}

# Instantiating WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.actionnetwork.com/nba/odds')

# Navigating to ML
ml_button = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[2]/div/div[1]/div[2]/select")
select = Select(ml_button)
select.select_by_visible_text('Moneyline')

# Getting table from HTML
html = driver.page_source
tables = pd.read_html(html)
odds = tables[0]
odds.dropna(inplace = True)
odds.reset_index(drop = True, inplace = True)

odds_df = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Home_Odds', 'Away_Odds'])
for index, row in odds.iterrows():

    # Retreiving teams
    teams = {}
    for key in team_map.keys():
        if row.Scheduled.find(key) != -1:
            teams[row.Scheduled.find(key)] = key
    keys = []
    for key in teams.keys():
        keys.append(key)
    if keys[0] > keys[1]:
        home_team = teams[keys[0]]
        away_team = teams[keys[1]]
    else:
        home_team = teams[keys[1]]
        away_team = teams[keys[0]]

    # Retreiving odds
    ml_string = row['Unnamed: 5']
    if len(ml_string) == 12:
        ml_string = ml_string.replace('ML', '')
        ml_away = ml_string[:4]
        ml_home = ml_string[-4:]
    if len(ml_string) == 13:
        ml_string = ml_string.replace('ML', '')
        if (ml_string[4] == '+') | (ml_string[4]=='-'):
            ml_away = ml_string[:4]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-4:]
    if len(ml_string) == 14:
        ml_string = ml_string.replace('ML', '')
        if (ml_string[5] == '+') | (ml_string[5]=='-'):
            ml_away = ml_string[:5]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-5:]
    try:
        ml_away = float(ml_away)
    except:
        continue
    try:
        ml_home = float(ml_home)
    except:
        continue
    series = pd.Series([home_team, away_team, ml_home, ml_away], index = odds_df.columns)
    odds_df = odds_df.append(series, ignore_index = True)
odds_df['Home_Prob'] = odds_df.Home_Odds.apply(calculate_odds)
odds_df['Away_Prob'] = odds_df.Away_Odds.apply(calculate_odds)
odds_df['Home_Team'] = odds_df.Home_Team.apply(lambda x: team_map[x])
odds_df['Away_Team'] = odds_df.Away_Team.apply(lambda x: team_map[x])

odds = odds_df




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


,Home_Team,Away_Team,Home_Odds,Away_Odds,Home_Prob,Away_Prob
0,Phoenix Suns,Los Angeles Clippers,-900.0,610.0,90.000000,14.084507
1,Atlanta Hawks,Cleveland Cavaliers,-138.0,118.0,57.983193,45.871560
2,Miami Heat,Dallas Mavericks,-260.0,200.0,72.222222,33.333333
3,Philadelphia 76ers,Boston Celtics,110.0,-130.0,47.619048,56.521739
4,Minnesota Timberwolves,Charlotte Hornets,-240.0,198.0,70.588235,33.557047
5,New Orleans Pelicans,Memphis Grizzlies,126.0,-158.0,44.247788,61.240310
6,Milwaukee Bucks,Indiana Pacers,-1200.0,750.0,92.307692,11.764706


In [26]:
capital = 100000
kelly = 10

# Merging dfs and adjusting column names
merged = pd.merge(fivethirtyeight_data, odds, on = ['Home_Team', 'Away_Team'])
merged.columns = ['Date', 'Away_Team', 'Away_Prob', 'Home_Team', 'Home_Prob', 'Home_Odds', 'Away_Odds', 
                 'Home_Prob_Implied', 'Away_Prob_Implied']

# Formatting columns
merged['Away_Prob'] = merged.Away_Prob.str.strip('%')
merged['Away_Prob'] = merged.Away_Prob.astype(float)
merged['Away_Prob'] = merged.Away_Prob/100
merged['Home_Prob'] = merged.Home_Prob.str.strip('%')
merged['Home_Prob'] = merged.Home_Prob.astype(float)
merged['Home_Prob'] = merged.Home_Prob/100
merged['Home_Prob_Implied'] = merged.Home_Prob_Implied/100
merged['Away_Prob_Implied'] = merged.Away_Prob_Implied/100

# Creating columns for bet size and potential payoff
merged['Home_Prob_Diff'] = merged.Home_Prob - merged.Home_Prob_Implied
merged['Away_Prob_Diff'] = merged.Away_Prob - merged.Away_Prob_Implied
merged['Home_Bet_Size'] = 0
merged['Away_Bet_Size'] = 0

# Calculating 538 bets and payoffs
for index, row in merged.iterrows():

    # Home Kelly Recommendation
    if row.Home_Prob_Diff < 0:
        merged.loc[index, 'Home_Bet_Size'] = 0
    else:
        p = row.Home_Prob
        q = 1.0 - p
        ml = row.Home_Odds
        if ml>=0:
            b = (ml/100)
        if ml<0:
            b = (100/abs(ml))
        kc = ((p*b) - q) / b
        bet = kc/kelly
        merged.loc[index, 'Home_Bet_Size'] = bet

    # Away Kelly Recommendation
    if row.Away_Prob_Diff < 0:
        merged.loc[index, 'Away_Bet_Size'] = 0
    else:
        p = row.Away_Prob
        q = 1.0 - p
        ml = row.Away_Odds
        if ml>=0:
            b = (ml/100)
        if ml<0:
            b = (100/abs(ml))
        kc = ((p*b) - q) / b
        bet = kc/kelly
        merged.loc[index, 'Away_Bet_Size'] = bet

# Creating bet columns
merged['Home_Bet'] = merged.Home_Bet_Size * capital
merged['Away_Bet'] = merged.Away_Bet_Size * capital

# Creating Payoff column
merged['Potential_Payoff'] = 0
for index, row in merged.iterrows():
    if row.Home_Bet > 0:
        if row.Home_Odds < 0:
            merged.loc[index, 'Potential_Payoff'] = (row.Home_Bet/abs(row.Home_Odds))*100
        if row.Home_Odds > 0:
            merged.loc[index, 'Potential_Payoff'] = row.Home_Bet * (row.Home_Odds/100)
    if row.Away_Bet > 0:
        if row.Away_Odds < 0:
            merged.loc[index, 'Potential_Payoff'] = (row.Away_Bet/abs(row.Away_Odds))*100
        if row.Away_Odds > 0:
            merged.loc[index, 'Potential_Payoff'] = row.Away_Bet * (row.Away_Odds/100)


In [36]:
capital_combined = 100000

# Calling in todays bets and diluting to necessary columns
bets = pd.read_csv('In_Season/Data/todays_bets.csv', index_col = 0)
bets = bets[['Away', 'Home', 'Home_Prob_Adjusted', 'Away_Prob_Adjusted']]
bets.columns = ['Away_Team', 'Home_Team', 'Home_Prob_Adjusted', 'Away_Prob_Adjusted']

# Merging and creating combined probs
combined = pd.merge(merged, bets, on = ['Home_Team', 'Away_Team'])
combined['Home_Prob_Combined'] = (combined.Home_Prob + combined.Home_Prob_Adjusted)/2
combined['Away_Prob_Combined'] = (combined.Away_Prob + combined.Away_Prob_Adjusted)/2

# Creating columns for bet recommendation
combined['Home_Prob_Diff_Combined'] = combined.Home_Prob_Combined - combined.Home_Prob_Implied
combined['Away_Prob_Diff_Combined'] = combined.Away_Prob_Combined - combined.Away_Prob_Implied
combined['Home_Bet_Size_Combined'] = 0
combined['Away_Bet_Size_Combined'] = 0

# Calculating combined bets and payoffs
for index, row in combined.iterrows():

    # Home Kelly Recommendation
    if row.Home_Prob_Diff_Combined < 0:
        combined.loc[index, 'Home_Bet_Size_Combined'] = 0
    else:
        p = row.Home_Prob_Combined
        q = 1.0 - p
        ml = row.Home_Odds
        if ml>=0:
            b = (ml/100)
        if ml<0:
            b = (100/abs(ml))
        kc = ((p*b) - q) / b
        bet = kc/kelly
        combined.loc[index, 'Home_Bet_Size_Combined'] = bet

    # Away Kelly Recommendation
    if row.Away_Prob_Diff_Combined < 0:
        combined.loc[index, 'Away_Bet_Size_Combined'] = 0
    else:
        p = row.Away_Prob_Combined
        q = 1.0 - p
        ml = row.Away_Odds
        if ml>=0:
            b = (ml/100)
        if ml<0:
            b = (100/abs(ml))
        kc = ((p*b) - q) / b
        bet = kc/kelly
        combined.loc[index, 'Away_Bet_Size_Combined'] = bet

# Creating bet columns
combined['Home_Bet_Combined'] = combined.Home_Bet_Size_Combined * capital_combined
combined['Away_Bet_Combined'] = combined.Away_Bet_Size_Combined * capital_combined

# Creating Payoff column
combined['Potential_Payoff_Combined'] = 0
for index, row in combined.iterrows():
    if row.Home_Bet_Combined > 0:
        if row.Home_Odds < 0:
            combined.loc[index, 'Potential_Payoff_Combined'] = (row.Home_Bet_Combined/abs(row.Home_Odds))*100
        if row.Home_Odds > 0:
            combined.loc[index, 'Potential_Payoff_Combined'] = row.Home_Bet_Combined * (row.Home_Odds/100)
    if row.Away_Bet_Combined > 0:
        if row.Away_Odds < 0:
            combined.loc[index, 'Potential_Payoff_Combined'] = (row.Away_Bet_Combined/abs(row.Away_Odds))*100
        if row.Away_Odds > 0:
            combined.loc[index, 'Potential_Payoff_Combined'] = row.Away_Bet_Combined * (row.Away_Odds/100)

,Date,Away_Team,Away_Prob,Home_Team,Home_Prob,Home_Odds,Away_Odds,Home_Prob_Implied,Away_Prob_Implied,Home_Prob_Diff,...,Away_Prob_Adjusted,Home_Prob_Combined,Away_Prob_Combined,Home_Prob_Diff_Combined,Away_Prob_Diff_Combined,Home_Bet_Size_Combined,Away_Bet_Size_Combined,Home_Bet_Combined,Away_Bet_Combined,Potential_Payoff_Combined
0,2022-02-15,Boston Celtics,0.45,Philadelphia 76ers,0.55,110.0,-130.0,0.476190,0.565217,0.073810,...,0.429547,0.560227,0.439773,0.084036,-0.125444,0.016043,0.000000,1604.325268,0.000000,1764.757795
1,2022-02-15,Cleveland Cavaliers,0.33,Atlanta Hawks,0.67,-138.0,118.0,0.579832,0.458716,0.090168,...,0.592141,0.538930,0.461070,-0.040902,0.002355,0.000000,0.000435,0.000000,43.503021,51.333565
2,2022-02-15,Dallas Mavericks,0.34,Miami Heat,0.66,-260.0,200.0,0.722222,0.333333,-0.062222,...,0.389923,0.635038,0.364962,-0.087184,0.031628,0.000000,0.004744,0.000000,474.425440,948.850880
3,2022-02-15,Charlotte Hornets,0.25,Minnesota Timberwolves,0.75,-240.0,198.0,0.705882,0.335570,0.044118,...,0.405608,0.672196,0.327804,-0.033687,-0.007766,0.000000,0.000000,0.000000,0.000000,0.000000
4,2022-02-15,Indiana Pacers,0.19,Milwaukee Bucks,0.81,-1200.0,750.0,0.923077,0.117647,-0.113077,...,0.118725,0.845637,0.154363,-0.077440,0.036716,0.000000,0.004161,0.000000,416.110423,3120.828169
5,2022-02-15,Memphis Grizzlies,0.61,New Orleans Pelicans,0.39,126.0,-158.0,0.442478,0.612403,-0.052478,...,0.752720,0.318640,0.681360,-0.123838,0.068957,0.000000,0.017791,0.000000,1779.088338,1126.005277
